In [ ]:
import pandas as pd
import numpy as np
import os

# Set input folder
###input_folder = "D:/MultiModal/Face_z_score/G_1/"

input_folder = r"C:\\Users\\user\\Desktop\\"

# 새롭게 다시 뽑아본 lean 개수. 
###output_folder = "D:/MultiModal/MultiModal_Model/results/face/face_landmark/G_lean_time/new_lean/"

output_folder = "C:\\Users\\user\\Desktop\\result\\"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Change working directory to input folder
os.chdir(input_folder)

def process_file(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name="Sheet1", index_col=0)
        if df.empty:
            return pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})
        
        # Calculate box size
        df["box.size"] = df["box.height"] * df["box.width"]

        # 5개의 열마다의 평균 값을 계산하고 있는 것.
        df["box.size.adj"] = df['box.size'].rolling(5, center=False).mean()
        df = df.iloc[5::5]

        # threshold의 경우, 
        #threshold = df["box.size.adj"].mean() * 0.05

        # Calculate delta
        df_delta = df['box.size.adj'].diff().iloc[1:]

        # Determine lean times
        #df_lean = np.where(df_delta.abs() > threshold, df_delta / df_delta.abs(), 0)
        
        
        df_lean = np.zeros(len(df_delta))
        for i in range(1, len(df_delta)):
            previous_mean = df["box.size.adj"].iloc[i-1]
            print("이전 프레임의 얼굴 크기는 : ", previous_mean)
            current_mean = df["box.size.adj"].iloc[i]
            print("다음 프레임의 얼굴 크기는 : ", current_mean)
            threshold = previous_mean * 0.05
            delta = current_mean - previous_mean

            if abs(delta) > threshold:
                df_lean[i] = np.sign(delta)
            else:
                df_lean[i] = 0
        
        
        def count_continuous_value(df, frame, d_range, d_threshold):
            if np.count_nonzero(df[frame:frame + d_range] == -1) >= d_threshold:
                df[frame:frame + d_range] = -1
                next_frame = frame + d_range
                print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : -1")
            elif np.count_nonzero(df[frame:frame + d_range] == 1) >= d_threshold:
                df[frame:frame + d_range] = 1
                next_frame = frame + d_range
                print(f"frame {(frame + 1) * 5} ~ frame {(frame + 1) * 5 + d_range * 5} : 1")
            else:
                df[frame] = 0
                next_frame = frame + 1
            return next_frame 
        
        # 첫 프레임부터 쭉 보고자, 0 으로 설정 
        frame = 0
        while frame < len(df_lean):
            frame = count_continuous_value(df_lean, frame, 5, 3)

        df_lean2_df = pd.DataFrame(df_lean, columns=['lean time'])

        total_seconds = np.arange(len(df_lean2_df)) * 0.03 ## 0.2초의 경우 5프레임. || 25 프레임의 경우 0.04
        minutes, seconds = divmod(total_seconds, 60)
        hours, minutes = divmod(minutes, 60)

        time_format = [f'{int(h):02}:{int(m):02}:{s:04.1f}' for h, m, s in zip(hours, minutes, seconds)]
        df_lean2_df.index = time_format

        return df_lean2_df

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})



file_name = f"Front_Test.xlsx"
file_path = os.path.join(input_folder, file_name)
output_file_name = f"Front_Test_lean.xlsx"
output_path = os.path.join(output_folder, output_file_name)

if os.path.exists(file_path):
    df = process_file(file_path)
    if df.empty:
        df = pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})
else:
    df = pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})

df.to_excel(output_path, index=True)




# 여러 개의 파일을 돌릴 때, 아래와 같이 코드 실행. 
# for week in ["1W","2W","3W","4W"]:
#     for group in ["G1","G2","G3","G4"]:
#         for step in ["S1", "S2", "S3"]:
#             file_name = f"Face_{week}_{group}_{step}.xlsx"
#             file_path = os.path.join(input_folder, file_name)
#             output_file_name = f"Face_{week}_{group}_{step}_lean.xlsx"
#             output_path = os.path.join(output_folder, output_file_name)

#             if os.path.exists(file_path):
#                 df = process_file(file_path)
#                 if df.empty:
#                     df = pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})
#             else:
#                 df = pd.DataFrame({"box.height": [0 for _ in range(6000)], "box.width": [0 for _ in range(6000)]})

#             df.to_excel(output_path, index=True)